Проверьте, встречаются ли в данных повторяющиеся наблюдения, и в качестве ответа укажите их количество. Если они есть, то удалите их из retail.

In [1]:
import pandas as pd

In [2]:
retail = pd.read_csv('C:/Users/spit/Desktop/karpov_1/KarpovCourses.Python/lesson_5/5_data.csv.zip', compression='zip', encoding='ISO-8859-1', parse_dates=['InvoiceDate'])

In [3]:
retail['InvoiceDate'] = pd.to_datetime(retail['InvoiceDate'])

In [4]:
retail_columns = retail.columns

In [5]:
retail_columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

Проверьте, встречаются ли в данных повторяющиеся наблюдения, и в качестве ответа укажите их количество. Если они есть, то удалите их из retail.

In [6]:
len(retail) - len(retail.drop_duplicates())

5268

In [7]:
retail = retail.drop_duplicates()

Сколько всего транзакций отменили пользователи? Считайте, что каждая строка - это отдельная транзакция.

In [8]:
retail['InvoiceNo'].str[0].unique()

array(['5', 'C', 'A'], dtype=object)

In [9]:
retail.loc[retail['InvoiceNo'].str[0] == 'C'].shape[0]

9251

Теперь отфильтруйте данные и оставьте в retail только те заказы, где Quantity > 0. В качестве ответа укажите число оставшихся строк.

In [10]:
retail = retail.query("Quantity > 0")

Посчитайте число заказов для каждого пользователя (CustomerID) из Германии (Germany). Оставьте только тех, кто совершил более N транзакций (InvoiceNo), где N – 80% процентиль. Запишите полученные id пользователей в germany_top (не весь датафрейм, только id).

In [11]:
invouce_dist = retail.query("Country == 'Germany'").groupby('CustomerID',as_index=False).agg(total_invoices = ('InvoiceNo','nunique'))

In [12]:
N = invouce_dist.total_invoices.quantile(0.8)

In [13]:
N

7.0

Это означает, что 80% значений в данных меньше или равны 7, а оставшиеся 20% больше.

In [14]:
germant_top = invouce_dist.query("total_invoices > @N")['CustomerID']

In [15]:
germant_top

3     12471.0
6     12474.0
8     12476.0
12    12481.0
16    12500.0
26    12524.0
37    12569.0
44    12600.0
51    12619.0
52    12621.0
56    12626.0
64    12647.0
69    12662.0
77    12705.0
78    12708.0
79    12709.0
81    12712.0
83    12720.0
Name: CustomerID, dtype: float64

Используя объект с id пользователей (germany_top), полученный на предыдущем шаге, отфильтруйте наблюдения и оставьте в данных записи только по интересующим нас юзерам. Результирующий датафрейм запишите в top_retail_germany.

In [16]:
top_retail_germany = retail.loc[retail.CustomerID.isin(germant_top)]

In [17]:
top_retail_germany.loc[top_retail_germany['StockCode']=='POST']

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1123,536527,POST,POSTAGE,1,2010-12-01 13:04:00,18.0,12662.0,Germany
6602,536967,POST,POSTAGE,1,2010-12-03 12:57:00,18.0,12600.0,Germany
6973,536983,POST,POSTAGE,1,2010-12-03 14:30:00,18.0,12712.0,Germany
9300,537197,POST,POSTAGE,3,2010-12-05 14:02:00,18.0,12647.0,Germany
9305,537198,POST,POSTAGE,1,2010-12-05 14:03:00,18.0,12647.0,Germany
...,...,...,...,...,...,...,...,...
535003,581179,POST,POSTAGE,1,2011-12-07 15:43:00,240.0,12471.0,Germany
535059,581183,POST,POSTAGE,4,2011-12-07 16:24:00,18.0,12569.0,Germany
535063,581184,POST,POSTAGE,2,2011-12-07 16:24:00,18.0,12569.0,Germany
537459,581266,POST,POSTAGE,5,2011-12-08 11:25:00,18.0,12621.0,Germany


Сгруппируйте top_retail_germany по коду товара (StockCode). Какой из продуктов добавляли в корзину чаще всего, кроме POST? 
Note: одним заказом считается единовременная покупка любого количества товара, т.е. без учета Quantity.


In [18]:
top_retail_germany.groupby('StockCode',as_index=False).size().sort_values('size',ascending=False).head(10)

,StockCode,size
1157,POST,213
409,22326,52
411,22328,38
453,22423,34
45,20719,30
709,22961,28
317,22077,26
514,22556,26
512,22554,25
607,22728,24


Вернемся к анализу полного датасета retail. Создайте колонку Revenue с суммой покупки, используя колонки Quantity и UnitPrice.

In [19]:
retail['Revenue'] = retail['Quantity'] * retail['UnitPrice']

In [20]:
retail

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,10.20
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60


Для каждой транзакции (InvoiceNo), посчитайте финальную сумму заказа. В качестве ответа укажите топ-5 (InvoiceNo) по сумме заказа (через запятую с пробелом и в порядке убывания TotalRevenue)

In [21]:
top_5_revenue = retail.groupby('InvoiceNo').agg(Total_revenue=('Revenue', 'sum')) \
                .sort_values('Total_revenue', ascending=False).head()

In [22]:
top_5_revenue

,Total_revenue
InvoiceNo,
581483,168469.60
541431,77183.60
574941,52940.94
576365,50653.91
556444,38970.00


In [23]:
top_5_revenue.index

Index(['581483', '541431', '574941', '576365', '556444'], dtype='object', name='InvoiceNo')